In [ ]:
# add the directory to the path, if not already added
import sys
import os
path_to_add = "/Users/vikra/Downloads/dryrun"  # Replace with the path to your directory
# Ensure the path is absolute
path_to_add = os.path.abspath(path_to_add)
if path_to_add not in sys.path:
  sys.path.append(path_to_add)

# main
import numpy as np
import numpy.linalg as la
import conjgrad
print(dir(conjgrad)) 

rPara=np.array([ [1e-14],      #rPara(1)=10^(-14)=eps
        [1e-9],       #rPara(2)=10^(-9)=epsPivot used in matrix factorization
        [1e-9],       #rPara(3)=10^(-6)=epsSD   10^(-3) is not good
        [0.5*1e-3],   #rPara(4)=0.5*10^(-3), MS9/9/19 relative error of f-value
        [1e-9],       # used in MiniQuadFunUnconstAdapt1.m and MiniQuadFunUnconstAdapt3 , if -df < rPara(5)*|fx0| , not acceptable progress
        [0.5],        # used in MiniQuadFunUnconstAdapt2.m  , if  -gdx*rPara(6) > -df , not acceptable progress  
        [1e18],       # maxDx, max value allowed for a step size in uSD,uPT,uCD
        [1e5],        #rparaFac0=1e+5, since 8/16/23
        [1e-1],       #iparaFac0=1e-1
] )   

iPara=np.array([  [0],        # iPara(1)=option used in LDLtOfSymMatrix21.m  for prearrangement
        [500],       # iPara(2)=max # of iterations for S-D alg
        [0],         # iPara(3)= MS:9/8/19 toDebug
        [21],        # iPara(4)= MS:9/9/19   #-1 is max length of printed out vector/matrix
        [3],         # iPara(5)=0 means use full recursion, iPara(5)=1 means use 1-recursion + steepest descent to solve instead of full recursion
        [0],         # iPara(6)=1 means use SD2 in the framework of v41
        [0],         # iPara(7)= MS:9/15/19 as option to pre-permutation of variables based on domain size
        [1],         # iPara(8)= MS:10/20/19 as option to check suff cond of optimality for earlier exit (1=y, 0=n)
        [0],         # iPara(9)= MS:10/31/19 as option to use 0=LDU (original/default), 1=QDQ' ([Q,D]=eig(A); Q=o.n. e.v. for symmetric case, enough!)
        [0],         # iPara(10)=MS:10/31/19 as option to use different choices of Xstart[]/Ystart[]: 0=Box(original/default), 1=space-Box(hybrid)
        [5],         # iPara(11)=VS:11/7/19, select bdy face 0=original, 1,2,3 for 3 versions of min dist from a line, 4 for min dist from a face approach 11/20
        [0],         # iPara(12)=tie breaker flag to select only one bdy face.
        [0],         # iPara(13)=as a flag controlling a pre-search procedure! It applies to: Sd,AllSd,PsdSd,
] )                

# Example matrix (symmetric for real eigenvalues and eigenvectors)
n=3
A = np.array([[4, 1, -2],          # hessian
                   [1, 2, -1],
                   [-2, -1, 1]])
d = np.array([1, 2, 3])    # linear term
#d=d.reshape(-1,1)
#x = np.zeros((n,1)) # starting point
x=np.zeros(n)
diagInv=1/np.diag(A)

# Perform spectral decomposition
diagD, Qmat = la.eigh(A)  # diagD is a vector of eigenvalues in ascending order
                               # Qmat is orthonormal matrix of corresponding eigenvectors
S={}   # create an empty dictionary
neig=sum(diagD>rPara[1])       # S['nNonZeroEig']=sum(diagD>rPara[1])
#print(diagD),print(Qmat[:,-2:])
Q=Qmat[:,n-neig:]                              # S['Q']=Qmat[:,n-S['nNonZeroEig']:]
D=diagD[n-neig:]           # S['D']=diagD[n-S['nNonZeroEig']:]
#D=D.reshape(-1,1)
invD=np.array(diagInv[n-neig:])                  #S['diagInv']=np.array(diagInv[n-S['nNonZeroEig']:])

print(Q)  
print(D)
print(neig)
print(invD)

isSoftStop=1
isTF=0
targetF=0
epsx, maxDx, epsg, nMax =rPara[1], rPara[6], rPara[2], iPara[1]

isNew, xnew, fxnew, niters, idExit=conjgrad.cg(n,A,d,Q,D,neig,x,epsx,maxDx,epsg,nMax,isSoftStop,isTF,targetF)
print("isNew:",isNew)
print("xnew:",xnew)
print("fxnew:",fxnew)
print("niters:",niters)
print("idExit:",idExit)
print("xols:" ,-(np.linalg.inv(A)@d))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'cg']
[[-0.43018908 -0.82119445]
 [ 0.90025898 -0.35949122]
 [-0.06686651  0.44317687]]
[1.59642438 5.51711323]
2
[0.5 1. ]
iter: 0
x: [0. 0. 0.]
g: [1. 2. 3.]
h: [-1. -2. -3.]
14.0
xnew: [-14. -28. -42.]
iter: 1
x: [-14. -28. -42.]
g: [  1. -26.  17.]
h: [ -70. -112. -224.]
966.0
xnew: [14.75 18.   50.  ]
iter: 2
x: [14.75 18.   50.  ]
g: [-22.     2.75   5.5 ]
h: [ -15.8125  -63.25   -126.5   ]
521.8125
xnew: [12.  7. 28.]
isNew: 1
xnew: [12.  7. 28.]
fxnew: 55.0
niters: 3
idExit: 0
xols: [12.  7. 28.]


In [1]:
import numpy as np
a=np.array([1, 2])
a=a.reshape(-1,1)
b=np.array([[3],
           [4]])
b=b.reshape(1,-1)
print(a,b)
print(np.inner(a,b))

[[1]
 [2]] [[3 4]]


ValueError: shapes (2,1) and (2,1) not aligned: 1 (dim 1) != 2 (dim 0)

In [1]:
import numpy as np
x=np.array([1, 2 ,3])
A = np.array([[4, 1, -2],          # hessian
                   [1, 2, -1],
                   [-2, -1, 1]])

print( np.dot(x.T,np.dot(A,x.T) ))
print( np.dot(x,np.dot(A,x) ))
print( x.T*(A*x) )
print( x@(A@x.T))

1
1
[[  4   4 -18]
 [  1   8  -9]
 [ -2  -4   9]]
1
